### Imports 

In [1]:
import model 
import data_proc_methods as dp
from importlib import reload 
import scanpy as sc
import torch
from sklearn.metrics import accuracy_score
from collections import Counter

In [8]:
import numpy as np

In [12]:
from importlib import reload

In [15]:
reload(dp)

<module 'data_proc_methods' from '/Users/fredrikstrom/Documents/KTH Dokument/scRNA_annotation-Thesis-/data_proc_methods.py'>

### Load data 

In [2]:
adata,encoder = dp.load_data('subchunk1_1.h5ad')

In [3]:
aloader = dp.create_annloader(adata)

In [5]:
bdata,_ = dp.load_data('subchunk1_2.h5ad')

In [6]:
bloader = dp.create_annloader(bdata)

In [ ]:
fdata,_ = dp.load_data('subchunk1_3.h5ad')

In [ ]:
floader = dp.create_annloader(fdata)

In [4]:
nn_model = model.BasicNeuralNetwork(adata.var['Gene'].values,adata.obs['supercluster_term'].cat.categories)

### Train model (multiple chunks from brain) 

In [5]:
model.train_nn(aloader,encoder,nn_model,label_type='sc',epochs=9)

Epoch:1, Loss:2.5271
Epoch:2, Loss:2.4991
Epoch:3, Loss:2.4947
Epoch:4, Loss:2.4908
Epoch:5, Loss:2.4892
Epoch:6, Loss:2.4894
Epoch:7, Loss:2.4901
Epoch:8, Loss:2.4976
Epoch:9, Loss:2.4887


In [ ]:
model.train_nn(bloader,encoder,nn_model,label_type='sc',epochs=5)

In [ ]:
model.train_nn(floader,encoder,nn_model,label_type='sc',epochs=5)

### Align test data and test the models accuracy 

In [6]:
cdata,_ = dp.load_data('Thalamic_Complex_RNA.h5ad')

In [16]:
map = dp.map_genes(adata.var['Gene'].values, cdata.var['Gene'].values)

In [17]:
cdata.obsm['aligned'] = dp.align(map,cdata.X)

/var/folders/6k/n1rxygz93j770tq1p431m1kw0000gn/T/ipykernel_59617/4016871610.py:1: ImplicitModificationWarning: Setting element `.obsm['aligned']` of view, initializing view as actual.
  cdata.obsm['aligned'] = dp.align(map,cdata.X)


In [18]:
c_pred = encoder.inverse_transform(nn_model.forward(torch.FloatTensor(cdata.obsm['aligned'].todense())).detach().numpy())

In [19]:
accuracy_score(y_pred=c_pred,y_true=cdata.obs['supercluster_term'].values)

0.9476227361587547